In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="LOGISTICREG")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

22/05/02 19:40:59 WARN Utils: Your hostname, herex resolves to a loopback address: 127.0.1.1; using 10.245.4.22 instead (on interface wlp58s0)
22/05/02 19:40:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 19:41:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/02 19:41:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#read the dataset
df=spark.read.csv('data.csv',inferSchema=True,header=True)

In [3]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- news: string (nullable = true)
 |-- final_manual_labelling: integer (nullable = true)



In [4]:
df.show(5)

+---+---------+--------------------+----------------------+
| id|     date|                news|final_manual_labelling|
+---+---------+--------------------+----------------------+
|  0|1/25/2022|Ripple announces ...|                     1|
|  1|1/25/2022|IMF directors urg...|                    -1|
|  2|1/25/2022|Dragonfly Capital...|                     1|
|  3|1/25/2022|Rick and Morty co...|                     0|
|  4|1/25/2022|How fintech SPACs...|                     0|
+---+---------+--------------------+----------------------+
only showing top 5 rows



In [5]:
from pyspark.ml.feature import Tokenizer
tokenization=Tokenizer(inputCol='news',outputCol='tokens')
tokenized_df=tokenization.transform(df)
tokenized_df.select(['id','date','tokens']).show(10,False)

+---+---------+---------------------------------------------------------------------------------------------------+
|id |date     |tokens                                                                                             |
+---+---------+---------------------------------------------------------------------------------------------------+
|0  |1/25/2022|[ripple, announces, stock, buyback,, nabs, $15, billion, valuation]                                |
|1  |1/25/2022|[imf, directors, urge, el, salvador, to, remove, bitcoin, as, legal, tender]                       |
|2  |1/25/2022|[dragonfly, capital, is, raising, $500, million, for, new, fund]                                   |
|3  |1/25/2022|[rick, and, morty, co-creator, collaborates, with, paradigm, on, nft, research, project]           |
|4  |1/25/2022|[how, fintech, spacs, lost, their, shine]                                                          |
|5  |1/25/2022|[multichain, vulnerability, put, a, billion, dollars, at,

In [6]:
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_df=stopword_removal.transform(tokenized_df)
refined_df.select(['id','date','refined_tokens']).show(10,False)

+---+---------+-------------------------------------------------------------------------------------+
|id |date     |refined_tokens                                                                       |
+---+---------+-------------------------------------------------------------------------------------+
|0  |1/25/2022|[ripple, announces, stock, buyback,, nabs, $15, billion, valuation]                  |
|1  |1/25/2022|[imf, directors, urge, el, salvador, remove, bitcoin, legal, tender]                 |
|2  |1/25/2022|[dragonfly, capital, raising, $500, million, new, fund]                              |
|3  |1/25/2022|[rick, morty, co-creator, collaborates, paradigm, nft, research, project]            |
|4  |1/25/2022|[fintech, spacs, lost, shine]                                                        |
|5  |1/25/2022|[multichain, vulnerability, put, billion, dollars, risk,, says, firm, found, bug]    |
|6  |1/25/2022|[youtube, wants, help, content, creators, capitalize, nfts]        

In [7]:
from pyspark.ml.feature import HashingTF,IDF
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')
hashing_df=hashing_vec.transform(refined_df)
hashing_df.select(['id','refined_tokens','tf_features']).show(4,False)


+---+-------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+
|id |refined_tokens                                                           |tf_features                                                                                                |
+---+-------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+
|0  |[ripple, announces, stock, buyback,, nabs, $15, billion, valuation]      |(262144,[18697,51983,64360,74682,97558,109858,110856,198843],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])            |
|1  |[imf, directors, urge, el, salvador, remove, bitcoin, legal, tender]     |(262144,[26144,41351,54828,65374,124004,132778,189902,214592,248887],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|2  |[dragonfly, capital, raising, $500, million, new, fund]

In [8]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [9]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)
tf_idf_df.select(['id','tf_idf_features']).show(4,False)

22/05/02 19:41:12 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |tf_idf_features                                                                                                                                                                                                                          |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |(262144,[18697,51983,64360,74682,97558,109858,110856,198843],[4.398555936625092,5.497168225293202,6.285625585657472,4.717009667743627,3.7679291130464807,3.212932270967353,5.592478405097527,7.201916317531627])                         |
|1  |(262144,[26144,41351,54828,65374,12

In [10]:
text_df=tf_idf_df.filter(((tf_idf_df.final_manual_labelling =='1') | (tf_idf_df.final_manual_labelling =='-1')| (tf_idf_df.final_manual_labelling =='0')))
text_df.groupBy('final_manual_labelling').count().show()

+----------------------+-----+
|final_manual_labelling|count|
+----------------------+-----+
|                    -1|  258|
|                     1| 1184|
|                     0| 1241|
+----------------------+-----+



In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = tf_idf_df.withColumn("token_count", len_udf(col('refined_tokens')))
refined_text_df.orderBy(rand()).show(10)

22/05/02 19:41:14 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+----+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|  id|      date|                news|final_manual_labelling|              tokens|      refined_tokens|         tf_features|     tf_idf_features|token_count|
+----+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-----------+
| 531|11/23/2021|Canadian investme...|                     0|[canadian, invest...|[canadian, invest...|(262144,[22371,46...|(262144,[22371,46...|         11|
|  51| 1/20/2022|Crypto staking fi...|                     1|[crypto, staking,...|[crypto, staking,...|(262144,[7132,553...|(262144,[7132,553...|         12|
|1037| 9/20/2021|Homeland Security...|                     1|[homeland, securi...|[homeland, securi...|(262144,[22732,31...|(262144,[22732,31...|          8|
| 980| 9/27/2021|Immutable X raise...|              

In [12]:
refined_text_df = refined_text_df.withColumn("Label", refined_text_df.final_manual_labelling.cast('float')).drop('final_manual_labelling')

In [13]:
refined_text_df.columns

['id',
 'date',
 'news',
 'tokens',
 'refined_tokens',
 'tf_features',
 'tf_idf_features',
 'token_count',
 'Label']

In [14]:
#Tenemos que crear dos modelos ,valor positivo y el resto, y valor negativo y el resto (1,0), y comparar los dos modelos (-1,0)
from pyspark.sql.functions import udf
from pyspark.sql.types import *

funct_negative_label = udf(lambda x: 1.00 if x == -1 else 0.00, FloatType())
func_positive_label = udf(lambda x: 1.00 if x == 1 else 0.00, FloatType())

refined_text_df = refined_text_df.withColumn("LabelNegative",funct_negative_label('Label'))
refined_text_df = refined_text_df.withColumn("LabelPositive",func_positive_label('Label'))

refined_text_df.show(5)



22/05/02 19:41:17 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+
| id|     date|                news|              tokens|      refined_tokens|         tf_features|     tf_idf_features|token_count|Label|LabelNegative|LabelPositive|
+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+
|  0|1/25/2022|Ripple announces ...|[ripple, announce...|[ripple, announce...|(262144,[18697,51...|(262144,[18697,51...|          8|  1.0|          0.0|          1.0|
|  1|1/25/2022|IMF directors urg...|[imf, directors, ...|[imf, directors, ...|(262144,[26144,41...|(262144,[26144,41...|          9| -1.0|          1.0|          0.0|
|  2|1/25/2022|Dragonfly Capital...|[dragonfly, capit...|[dragonfly, capit...|(262144,[55307,82...|(262144,[55307,82...|          7|  1.0|          0.0|          1.0

In [15]:
# Veamos si las dos matrices generadas cumplen las caracteristicas que queriamos

In [16]:
#Ponemos en el filtro todos los casos 1,0,-1 para ver que las negativas no tenga ningun 1, y las positivas ningun -1
text_df_positivo=refined_text_df.filter(((refined_text_df.LabelPositive =='1') | (refined_text_df.LabelPositive =='-1')| (refined_text_df.LabelPositive =='0')))
text_df_negativo=refined_text_df.filter(((refined_text_df.LabelNegative =='1') | (refined_text_df.LabelNegative =='-1')| (refined_text_df.LabelNegative =='0')))


In [17]:
text_df_negativo.groupBy('LabelNegative').count().show()

+-------------+-----+
|LabelNegative|count|
+-------------+-----+
|          1.0|  258|
|          0.0| 2425|
+-------------+-----+



In [18]:
text_df_positivo.groupBy('LabelPositive').count().show()

+-------------+-----+
|LabelPositive|count|
+-------------+-----+
|          1.0| 1184|
|          0.0| 1499|
+-------------+-----+



In [19]:
from pyspark.ml.feature import VectorAssembler

df_assembler = VectorAssembler(inputCols=['tf_idf_features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(refined_text_df)

In [20]:
model_text_df.show(5)


22/05/02 19:41:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+--------------------+
| id|     date|                news|              tokens|      refined_tokens|         tf_features|     tf_idf_features|token_count|Label|LabelNegative|LabelPositive|        features_vec|
+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+--------------------+
|  0|1/25/2022|Ripple announces ...|[ripple, announce...|[ripple, announce...|(262144,[18697,51...|(262144,[18697,51...|          8|  1.0|          0.0|          1.0|(262145,[18697,51...|
|  1|1/25/2022|IMF directors urg...|[imf, directors, ...|[imf, directors, ...|(262144,[26144,41...|(262144,[26144,41...|          9| -1.0|          1.0|          0.0|(262145,[26144,41...|
|  2|1/25/2022|Dragonfly Capital...|[dragonfly, capit...|[dr

In [21]:
from pyspark.ml.classification import LogisticRegression
#split the data 
training_df_negative,test_df_negative=model_text_df.randomSplit([0.75,0.25])
#split the data 
training_df_positive,test_df_positive=model_text_df.randomSplit([0.75,0.25])

In [22]:
log_reg_positive = LogisticRegression(featuresCol='features_vec',labelCol='LabelPositive').fit(training_df_positive)
log_reg_negative = LogisticRegression(featuresCol='features_vec',labelCol='LabelNegative').fit(training_df_negative)



22/05/02 19:41:24 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:26 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/02 19:41:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/05/02 19:41:28 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:30 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:31 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:32 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:33 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:34 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/02 19:41:36 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/05/

In [ ]:
log_reg_negative.write().save("./model_neg")
log_reg_positive.write().save("./model_pos")


In [ ]:
# model_neg = LogisticRegression.load("model_neg")
# model_pos = LogisticRegression.load("model_pos")

In [24]:
results_positive = log_reg_positive.evaluate(test_df_positive).predictions
results_negative = log_reg_negative.evaluate(test_df_negative).predictions

/home/herex/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [25]:
results_positive.show(5)

22/05/02 19:43:16 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB


+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+--------------------+--------------------+--------------------+----------+
| id|     date|                news|              tokens|      refined_tokens|         tf_features|     tf_idf_features|token_count|Label|LabelNegative|LabelPositive|        features_vec|       rawPrediction|         probability|prediction|
+---+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+-------------+-------------+--------------------+--------------------+--------------------+----------+
| 11|1/25/2022|Twitter is growin...|[twitter, is, gro...|[twitter, growing...|(262144,[1512,480...|(262144,[1512,480...|          5|  1.0|          0.0|          1.0|(262145,[1512,480...|[15.7481994773413...|[0.99999985524159...|       0.0|
| 13|1/24/2022|Walmart director ...|

In [26]:
#confusion matrix positive model
true_postives = results_positive[(results_positive.Label == 1) & (results_positive.prediction == 1)].count()
true_negatives = results_positive[(results_positive.Label == 0) & (results_positive.prediction == 0)].count()
false_positives = results_positive[(results_positive.Label == 0) & (results_positive.prediction == 1)].count()
false_negatives = results_positive[(results_positive.Label == 1) & (results_positive.prediction == 0)].count()

22/05/02 19:43:51 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:43:53 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:43:54 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:43:55 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB


In [27]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.6589403973509934


In [28]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.6958041958041958


In [29]:
accuracy=float((true_postives+true_negatives) /(results_positive.count()))
print(accuracy)

22/05/02 19:44:14 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


0.6153846153846154


In [30]:
#confusion matrix negative model
true_postives = results_negative[(results_negative.Label == 1) & (results_negative.prediction == 1)].count()
true_negatives = results_negative[(results_negative.Label == 0) & (results_negative.prediction == 0)].count()
false_positives = results_negative[(results_negative.Label == 0) & (results_negative.prediction == 1)].count()
false_negatives = results_negative[(results_negative.Label == 1) & (results_negative.prediction == 0)].count()

22/05/02 19:44:19 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:44:20 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:44:22 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB
22/05/02 19:44:24 WARN DAGScheduler: Broadcasting large task binary with size 20.6 MiB


In [31]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.027777777777777776


In [32]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.3333333333333333


In [33]:
accuracy=float((true_postives+true_negatives) /(results_positive.count()))
print(accuracy)

22/05/02 19:44:25 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


0.44461538461538463


22/05/02 22:37:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2897258 ms exceeds timeout 120000 ms
22/05/02 22:37:10 WARN SparkContext: Killing executors is not supported by current scheduler.
